# Exercise sheet 2
### Due 11/11/2022

___

### Group:
-  
-  
- 

___

## Problem 1 - Prime counting

Write a function `count_primes` that, given an input integer `n`, calculates the number of prime numbers smaller or equal to `n`.

Use a double loop for this.

In [66]:
def count_primes(n):
    '''Returns the number of prime numbers netween 1 and n.'''
    number_of_primes = 0
    for i in range(2, n + 1): # 2 is the lowest prime number, and we have to 
                              # include n to check whether it is prime 
                              
        has_dividends = 0
        for j in range(2,i):  # now for each i, we check if there is any
                              # number from 2 up to i-1 by which i can be 
                              # divided. Here we don't include i itself, 
                              # because any number can be divided by itself
            if i % j == 0:
                has_dividends = 1 # once we find the first dividend,
                                  # we stop the inner loop with `break`
                break
        # Now we're out of the inner loop over j
        if has_dividends == 0:
            number_of_primes += 1   # if there wasn't even one j such that 
                                    # i%j == 0, then i is prime. Notice that 
                                    # this if-statement will repeat for all 
                                    # values of i from 2 to up to n

    return number_of_primes

# Always test your function near the boundaries!
print(count_primes(1))    # 1 is not a prime number, so we expect 0
print(count_primes(2))    # 2 is the lowest prime number, so we expect 1
print(count_primes(3))    # Here we expect 2
print(count_primes(1000)) # Here we expect 168 (Yes, I checked)

0
1
2
168


## Problem 2 - Matrix multiplication 

This exercise is about 2D matrices, such as

$$A = \begin{bmatrix} 5 & 10 & 2 & 1 \\ 3 & 0 & 2 & 1\\ 0.5 & 2 & 1 & 6\end{bmatrix}, B = \begin{bmatrix} 3 & 3 & 1 \\ 2 & 5 & 3\\ 0.1 & 20 & 2\\ 0 & 4 & 1\end{bmatrix}, C = \begin{bmatrix} 1 & 0.5 \\ 4 & 3\end{bmatrix}$$

The simplest way to represent a matrix in Pyhon is using nested lists.

For example, matrix $C$ above can be represented by a list `cmat = [[1, 0.5], [4,3]]`. This way, each element of the list is a row of the matrix, and so element $C_{i,j}$ can be accessed through `cmat[i][j]`.

Using nested loops, write a function `multiply2d` that accepts as input two matrices in the form of nested lists, calculates their product and returns it also in the form of a nested list.

Test your function by computing $A\times B$ and $C^2$.

In case you need a reminder- the product of two matrices $P \equiv A\times B$ is such that $P_{i,j} = \sum_{k=1}^n A_{i,k}B_{k,j}$.
Here, $n$ is the number of columns of matrix $A$, which has to be the same as the number of rows of matrix $B$.

In [53]:
def multiply2d(mat1, mat2):
    '''Multiplies two 2D matrices in the form of nested lists.'''
    
    nrows1, nrows2 = len(mat1), len(mat2)
    ncols1, ncols2 = len(mat1[0]), len(mat2[0])
    
    if ncols1 != nrows2:
        print("[multiply2d]Error: Wrong shapes for multiplication!")
        return None  # This ends the function if the input matrix shapes 
                     # are incompatible
    
    prod = []
    for i in range(nrows1):
        prod.append([])  # for every row in mat1, we append 
                         # a new row to our product matrix 
        for j in range(ncols2):
            res = 0 
            for k in range(ncols1): # this loop performs the sum
                res += mat1[i][k] * mat2[k][j]
            prod[i].append(res)  # We now append the resulting sum 
                                 # to the row of `prod` that we are
                                 # currently working on
        
    return prod

### Another kind of solution that some of you found was to create first an array with the final shape flled with zeros, and then add the products one by one:

In [56]:
def multiply2d_version2(mat1, mat2):
    '''Multiplies two 2D matrices in the form of nested lists.
    This is an alternative solution compared to multiply2d.
    '''
    
    nrows1, nrows2 = len(mat1), len(mat2)
    ncols1, ncols2 = len(mat1[0]), len(mat2[0])
                                                                  
    if ncols1 != nrows2:
        print("[multiply2d]Error: Wrong shapes for multiplication!")
        return None      

    prod = [[0 for i in range(nrows1)] for j in range(ncols2)] # This is a matrix with only
                                                               # zeros, which has the right
                                                               # shape of the output matrix:
                                                               # (nrows1 x ncols2)
    for i in range(nrows1):
        for j in range(ncols2):
            res = 0 
            for k in range(ncols1): # this loop performs the sum
                prod[i][j] += mat1[i][k] * mat2[k][j]
            
    return prod

In [65]:
# Test functions:

amat = [[5, 10, 2, 1], [3, 0, 2, 1], [0.5, 2, 1, 6]]
bmat = [[3, 3, 1], [2, 5, 3], [0.1, 20, 2], [0, 4, 1]]
cmat = [[1, 0.5], [4, 3]]

print(multiply2d(amat, bmat)) # A * B
print(multiply2d(cmat, cmat)) # C ** 2


[[35.2, 109, 40], [9.2, 53, 8], [5.6, 55.5, 14.5]]
[[3.0, 2.0], [16, 11.0]]


## Problem 3 - Nuclear iterations

In this problem you'll be organizing nuclear data tables into a nested dictionary.

A nucleus $^\rm{A}_\rm{Z}\rm{X}$ is characterized by its number of protons $\rm{Z}$, also called the *atomic number*, and the total number of protons and neutrons $\rm{A}$, called the *mass number*. The name of the element, as well as its chemical symbol $X$, depend only on $Z$. Nuclei with the same number of protons $Z$ but different mass numbers $A$ are called isotopes. Some isotopes are stable, others are unstable.

Each nuclear isotope can be uniquely idenfified by a "particle code" $\rm{A}\times100+\rm{Z}$. So in the case of $^{28}_{14}\rm{Si}$, which has 14 protons and 14 protons, this particle code is `28 * 100 + 14 == 2814`.

In this exercie you'll use the lists `database` and `symbols`, both loaded in the cell below:

In [5]:
from pickle import load as pload

with open("ex-02_nuc_database.pkl", "rb") as f: # Make sure this pkl file is in the same directory
                                                # as this notebook
    database, symbols = pload(f)

`database` is a table (adopted from <href>https://www-nds.iaea.org/amdc/</href>), given here in the form of a list of lists. Each of these lists has information about a different nuclear isotope: particle code, half-life in seconds, and the year its spectrum was measured. If the half-life is `None`, the isotope is stable.

`symbols` is a simple list of strings with chemical symbols of the elements ordered by $Z$.

Oragnize the information in `database` in the form of a **nested dictionary** called `nuc_dict`.

For every chemical element, `nuc_dict` should map its chemical symbol to a dictionary with the following information: 
- the number of protons $Z$;
- how many stable and unstable isotopes of the element are present in the database;
- the minimum and maximum number of neutrons of the isotopes in the database;
- the average of the half-lives of its unstable isotopes;
- year when the first isotope was discovered

In the end, your dictionary shuld look something like:

```
nuc_dict = {"H" :  {"n_protons"        : 1,
                    "n_neutrons_min"   : 0,      # hydrogen-1 has no neutrons
                    "n_neutrons_max"   : 5,      # hydrogen-6 has 5 neutrons
                    "n_iso_stable"     : 2,      # hydrogen-1 and hydrogen-2 are stable
                    "n_iso_unstable"   : 4,      # hydrogen-3 through hydrogen-6 are unstable
                    "average_halflife" : 337.83, # average of the half-lives of the
                                                 # four unstable hydrogen isotopes
                    "first_measured" : 1920},    # hydrogen-1 measured 1920 (other isotopes later)
            "He" : {...},
            ...
           }
```

In [60]:
%%time

zset = range(1, len(symbols)) # We start at Z == 1 with protons 
                              # (Z == 0 would be just a neutron)

nuc_dict = {}

                                       ###################################
for pcode, halflife, year in database: ### Notice how I run only **once** 
                                       ### through my long database!
                                       ### This is extremely important for
                                       ### having an efficient parser.
                                       ###################################
    z = pcode % 100
    a = pcode // 100
    n = a - z         # number of neutrons of the isotope
    sym = symbols[z]
    
    if sym not in nuc_dict.keys():                # If it's the first occurrence of the
                                                  # element, we create a new entry
        nuc_dict[sym] = {'n_protons'        : z,
                         'n_neutrons_min'   : n,
                         'n_neutrons_max'   : n,
                         'first_measured'   : year,
                         'n_iso_stable'     : 0,
                         'n_iso_unstable'   : 0,
                         'average_halflife' : 0
                        }
    else:
        if nuc_dict[sym]['n_neutrons_max'] < n:
            nuc_dict[sym]['n_neutrons_max'] = n
    
        if nuc_dict[sym]['n_neutrons_min'] > n:
            nuc_dict[sym]['n_neutrons_min'] = n
        
        if year is not None:
            if nuc_dict[sym]['first_measured'] is None: # Note that `None` cannot be 
                                                        # compared using `>` or `<`, or 
                                                        # it will result in an error
                nuc_dict[sym]['first_measured'] = year
                
            elif nuc_dict[sym]['first_measured'] > year:
                nuc_dict[sym]['first_measured'] = year
                
    # Now we take case of the half life and the stable/unstable distinction:
                
    if halflife:                                      # This evaluates to True if halflife > 0
                                                      # (isotope is unstable)
                                                      # and to False if halflife is None 
                                                      # (isotope is stable)
        nuc_dict[sym]['n_iso_unstable'] += 1
        nuc_dict[sym]['average_halflife'] += halflife # For now we collect the sum of the
                                                      # half-lives of all unstable isotopes
    else:
        nuc_dict[sym]['n_iso_stable'] += 1
        
for sym in nuc_dict.keys():
    nuc_dict[sym]['average_halflife'] /= nuc_dict[sym]['n_iso_unstable'] # Finally, we calculate 
                                                                         # the average half-life 
    

CPU times: user 11.4 ms, sys: 1.08 ms, total: 12.4 ms
Wall time: 11.8 ms


In [7]:
nuc_dict['H']

{'n_protons': 1,
 'n_neutrons_min': 0,
 'n_neutrons_max': 5,
 'first_measured': 1920,
 'n_iso_stable': 2,
 'n_iso_unstable': 4,
 'average_halflife': 337.83}

In [8]:
nuc_dict['Y']

{'n_protons': 39,
 'n_neutrons_min': 38,
 'n_neutrons_max': 70,
 'first_measured': 1923,
 'n_iso_stable': 1,
 'n_iso_unstable': 66,
 'average_halflife': 146.35436363636362}